lora 어댑터와 기존 프루닝된 모델 업로드 후 결합하여 저장

In [ ]:
from peft import PeftModel
import torch
from model_loader import load_pruned_model

model_path = "/pruned_activation"
base_model, tokenizer = load_pruned_model(
    model_path,
    device=torch.device("cuda")
)
full_model = PeftModel.from_pretrained(base_model, "/lora_tomato/checkpoint-921")

full_model = full_model.merge_and_unload()

# full model 저장
full_model.save_pretrained("/full_model")
tokenizer.save_pretrained("/full_model")

gguf 파일 생성 및 4bit 양자화 적용 (로컬에서는 아래 명령어 느낌표 제거 후 터미널에서 순차적으로 진행하면 됨)

In [ ]:
# gguf 파일 생성
!git clone https://github.com/ggerganov/llama.cpp.git

%cd /content/llama.cpp
!pip install -r requirements.txt

!mkdir -p build
!cmake -B build
!cmake --build build

!python convert_hf_to_gguf.py /full_model --outfile /quantized_tomato.gguf --outtype f32

# 양자화 적용
!./build/bin/llama-quantize /quantized_tomato.gguf /quantized_tomatoQ4_K_M.gguf Q4_K_M